In [1]:
#Preliminary predictivemodel for clothing predictions

In [2]:
import numpy as np
import pandas as pd
import random
import datetime
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier

In [3]:
#Creating the dataset

#features

"""
Weather features:
1. Temperature (Celcius) (-20 to 25)
2. Humidity (%) (relative) (0 to 100)
3. Probability of Precipitation (%) (0 to 100)
4. Wind speed (mph)

User features:
1. Age (15 to 60)
2. Weight (kg)
3. Sex
4. Height (cm)
5. Fat percentage
6. BMI
"""

#output
"""
output will be in the form of a boolean array with each index presenting different clothing articles
0. short sleve shirt + long pants (always on)
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

'\noutput will be in the form of a boolean array with each index presenting different clothing articles\n0. short sleve shirt + long pants (always on)\n1. Thermal (both top and bottom)\n2. Jacket/Hoodie\n3. light downjacket\n4. Thick down/Winter down\n5. Wind breaker\n6. Umbrella/Raincoat\n'

In [4]:
#smart generator for people's height, weight, age, sex
def peopledatagenerator():
    sex = random.randint(0,1)
    age = random.randint(15, 60)
    #stats from (United states)
    #https://en.wikipedia.org/wiki/Average_human_height_by_country
    #https://en.wikipedia.org/wiki/Human_body_weight
    #male
    if sex == 0:
        height = np.random.normal(loc = 175.3, scale = 6.35)
        weight = np.random.normal(loc = 90.6, scale = 15)
    #female
    else:
        height = np.random.normal(loc = 161.3, scale = 5.59)
        weight = np.random.normal(loc = 77.5, scale = 15)
        
    return sex, age, height, weight




In [5]:
from datetime import date
data = []
currentdate = date.today()
for i in range(100000):
    date = (currentdate - datetime.timedelta(i)).strftime("%d/%m/%Y") #display in day/month/year
    temperature = random.randint(0, 45) - 20
    humidity = random.randint(0, 100)
    precipitation = random.randint(0, 100)
    windspeed = random.randint(0, 40) #mph
    #age = random.randint(15, 60)
    #weight = random.randint(40, 120) #kg
    #height = random.randint(110, 200) #cm
    #sex = random.randint(0,1) # 0:M 1:F
    
    
    sex, age, height, weight = peopledatagenerator()

    #fat percentage formula from
    #https://www.gaiam.com/blogs/discover/how-to-calculate-your-ideal-body-fat-percentage
    
    BMI = weight/(height**2)*10000
    if sex == 0:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 16.2
        
    elif sex == 1:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 5.4
    
    #adjustment for negative fatpercentage
    if fatpercentage < 0:
        fatpercentage = 1
        
    datapoint = (date, temperature, humidity, precipitation, windspeed, age, weight, height, sex, fatpercentage, BMI)
    data.append(datapoint)
    

In [6]:
columnstring = 'date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi'
column = columnstring.split(',')
df = pd.DataFrame(data, columns = column)

In [7]:
df[:100]

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi
0,12/01/2022,-14,86,57,30,52,108.597568,175.889426,0,37.883251,35.102709
1,11/01/2022,3,42,23,9,26,79.699237,163.126371,1,36.520744,29.950620
2,10/01/2022,6,13,56,7,39,84.996828,160.850646,1,42.991972,32.851643
3,09/01/2022,-11,41,29,30,26,72.087603,170.433302,1,30.360564,24.817136
4,08/01/2022,-12,51,99,1,58,81.148904,154.912303,1,48.518135,33.815113
...,...,...,...,...,...,...,...,...,...,...,...
95,09/10/2021,-10,78,88,28,21,63.521086,174.551434,1,24.447984,20.848320
96,08/10/2021,-7,22,84,24,22,66.822574,174.803792,0,15.102352,21.868626
97,07/10/2021,17,3,0,27,40,49.501524,167.060145,1,25.084042,17.736702
98,06/10/2021,19,21,34,9,35,96.910248,169.315636,0,32.415499,33.804582


In [8]:
#layers

"""
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

#cold resistance calculation

male_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}


female_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}

# cold resistance categories:
# low: 0 - 24
# mid-low: 25 - 49
# mid-high: 50 - 74
# high: 75 - 99
cold_resistance_scores = {
    'low' : 25,
    'mid-low' : 50,
    'mid-high' : 75,
    'high' : 100
}




In [9]:
#returns cold resistance score
#formula:

#cold_resistance = age_score * gender_age_weight + bmi_score + gender_bmi_weight + gender_fatpercentage_weight
def coldresistance(sex, age, fatpercentage, BMI):
    #female case
    if sex == 1:
        if (age <= 15 or age >= 50):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 16 <= age <= 25:
            age_score = cold_resistance_scores['mid-high']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-low']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 14):
            fatpercentage_score = cold_resistance_scores['low']
        elif (14 < fatpercentage <= 20):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (20 < fatpercentage <= 25): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
      
    #men
    else:
        if (age <= 12 or age >= 50):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 16 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 8):
            fatpercentage_score = cold_resistance_scores['low']
        elif (8 < fatpercentage <= 15):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (15 < fatpercentage <= 24): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
            
    #female case
    if sex == 1:
        cold_resistance = age_score/100*female_weights['age'] + bmi_score/100*female_weights['bmi'] + fatpercentage_score/100*female_weights['fatpercentage']
    
    #male case
    elif sex == 0:
        cold_resistance = age_score/100*male_weights['age'] + bmi_score/100*male_weights['bmi'] + fatpercentage_score/100*male_weights['fatpercentage']
    
    return cold_resistance

In [10]:
df['cold_resistance'] = df.apply(lambda row : coldresistance(row['sex'], row['age'], row['fatpercentage'], row['bmi']), axis = 1)

In [11]:
df.head()

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance
0,12/01/2022,-14,86,57,30,52,108.597568,175.889426,0,37.883251,35.102709,70.0
1,11/01/2022,3,42,23,9,26,79.699237,163.126371,1,36.520744,29.950620,100.0
2,10/01/2022,6,13,56,7,39,84.996828,160.850646,1,42.991972,32.851643,100.0
3,09/01/2022,-11,41,29,30,26,72.087603,170.433302,1,30.360564,24.817136,92.5
4,08/01/2022,-12,51,99,1,58,81.148904,154.912303,1,48.518135,33.815113,70.0


In [12]:
clothesmap = {
    'thermal' : 0, #(both top and bottom)
    'hoodie' : 1,
    'fleece' : 2,
    'wool' : 3,
    'light_down' : 4,
    'thick_down' : 5,
    'wind_breaker' : 6,
    'umbrella' : 7,
    'winter_boots': 8
}

heatmap = {
    'thermal' : 30, #(both top and bottom)
    'hoodie' : 10,
    'fleece' : 20,
    'wool' : 40,
    'light_down' : 50,
    'thick_down' : 60
}



In [13]:
#rules
"""
1. If temp < 5 : Thermal = True and thickdown = True
2. If temp < 15 : Jacket = True
3. If temp < 10 : lightdown = True
4. If windspeed >= 25 : windbreaker = True
5. If precipitation > 50%: umbrella/raincoast = True


cold resistance categories:
low: 0 - 24
mid-low: 25 - 49
mid-high: 50 - 74
high: 75 - 99
"""



def clothingPredictor(coldresistance, temperature, humidity, precipitation, windspeed):
    #prediction is an array of booleans with each index corresponding to a certain clothing article
    prediction = [False] * len(clothesmap.keys())
    
    
    #wind breaker and umbrella are independent of cold resistance
    if windspeed >= 25:
        prediction[clothesmap['wind_breaker']] = True
    
    if precipitation >= 50:
        prediction[clothesmap['umbrella']] = True
    
    
    #low cold resistance
    if coldresistance <= 25:
        #print('low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        elif temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        #if temp > 25 dont put on any additional layers
        else:
            pass
            
    #mid-low cold resistance
    elif 25 < coldresistance <= 50:
        #print('mid- low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 25 dont put on any additional layers
        else:
            pass
        
    #mid-high cold resistance
    elif 50 < coldresistance <= 75:
        #print('mid - high cr')
        if 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 20 dont put on any additional layers
        else:
            pass
        
    #high cold resistance
    elif coldresistance > 75:
        #print('high cr')
        if 10 <= temperature <= 15:
            prediction[clothesmap['hoodie']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif -10 < temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= -10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            #print('yay')
        #if temp > 15 dont put on any additional layers
        else:
            pass

        return prediction

In [14]:
def findlowestheatscore(map):
    lowest = 100000
    for i in map:
        if map[i] < lowest:
            lowest = map[i]
    
    return lowest

#temperature range is from -20 to 25
def predictusingheatscore(temperature, cold_resistance, heatmap, windspeed, precipitation):
    #print(temperature)
    prediction = [False] * len(clothesmap.keys())
    
    #wind breaker and umbrella are independent of cold resistance
    if windspeed >= 25:
        prediction[clothesmap['wind_breaker']] = True
    
    if precipitation >= 50:
        prediction[clothesmap['umbrella']] = True
        
    if precipitation >= 50 and temperature <= 5:
        prediction[clothesmap['winter_boots']] = True
    
    #calulation layers required
    lowest_heatscore = findlowestheatscore(heatmap)

    
    #map using linear equation (-20,300) to (25, 0), x is temperature y is insulation needed
    insulation_calculation = -1 * (300/45) * temperature + (-1) * (300/45)* (-25)
    
    insulation_required = insulation_calculation - cold_resistance
    
    if insulation_required < lowest_heatscore:
        #dont need put on any extra layers if insulation required is lower than the lowest_heatscore
        return prediction
    
    else:
        result = []
        
        while insulation_required > 0:
            smallest_difference = 100000
            best_cloth = ''
            #hardstop at 5 layers
            if len(result) == 6:
                break
            
            for clothes in heatmap:
                #avoid repeats
                if clothes in result: continue
                difference = abs(insulation_required - heatmap[clothes])
                if difference < smallest_difference:
                    smallest_difference = difference
                    best_cloth = clothes
            
            result.append(best_cloth)
            insulation_required = insulation_required - heatmap[best_cloth]
            
        for ite in result:
            prediction[clothesmap[ite]] = True
            
        return prediction
                
                
            
            
        

In [15]:
print(predictusingheatscore(20, 10, heatmap, 25, 40))

[False, True, True, False, False, False, True, False, False]


In [16]:
"""
thermal < fleece/wool < hoodie < light_down < thick_down


add shoes/winter shoes

add more dimensions to calculate the score





"""

'\nthermal < fleece/wool < hoodie < light_down < thick_down\n\n\nadd shoes/winter shoes\n\nadd more dimensions to calculate the score\n\n\n\n\n\n'

In [17]:
for article in clothesmap:
    df[article] = df.apply(lambda row : predictusingheatscore(row['temperature'], row['cold_resistance'], heatmap,\
                                                              row['windspeed'], row['precipitation'])[clothesmap[article]], axis = 1)

In [18]:
# for article in clothesmap:
#     df[article] = df.apply(lambda row : clothingPredictor(row['cold_resistance'], row['temperature'], row['humidity'],\
#                                                           row['precipitation'], row['windspeed'])[clothesmap[article]], axis = 1)

In [19]:
df.head()

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,...,cold_resistance,thermal,hoodie,fleece,wool,light_down,thick_down,wind_breaker,umbrella,winter_boots
0,12/01/2022,-14,86,57,30,52,108.597568,175.889426,0,37.883251,...,70.0,True,True,False,True,True,True,True,True,True
1,11/01/2022,3,42,23,9,26,79.699237,163.126371,1,36.520744,...,100.0,False,False,False,False,True,False,False,False,False
2,10/01/2022,6,13,56,7,39,84.996828,160.850646,1,42.991972,...,100.0,True,False,False,False,False,False,False,True,False
3,09/01/2022,-11,41,29,30,26,72.087603,170.433302,1,30.360564,...,92.5,False,False,False,True,True,True,True,False,False
4,08/01/2022,-12,51,99,1,58,81.148904,154.912303,1,48.518135,...,70.0,True,False,False,True,True,True,False,True,True


In [20]:
check =  df[['temperature', 'humidity', 'precipitation', 'windspeed', 'cold_resistance', 'age', 'fatpercentage', 'bmi', 'cold_resistance' ]]

In [21]:
features = df[['temperature', 'humidity', 'precipitation', 'windspeed', 'age', 'weight', 'height', 'sex', 'fatpercentage', 'bmi', 'cold_resistance' ]]

In [22]:
features.head()

,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance
0,-14,86,57,30,52,108.597568,175.889426,0,37.883251,35.102709,70.0
1,3,42,23,9,26,79.699237,163.126371,1,36.520744,29.950620,100.0
2,6,13,56,7,39,84.996828,160.850646,1,42.991972,32.851643,100.0
3,-11,41,29,30,26,72.087603,170.433302,1,30.360564,24.817136,92.5
4,-12,51,99,1,58,81.148904,154.912303,1,48.518135,33.815113,70.0


In [23]:
output = df[clothesmap.keys()]

In [24]:
output.head()

,thermal,hoodie,fleece,wool,light_down,thick_down,wind_breaker,umbrella,winter_boots
0,True,True,False,True,True,True,True,True,True
1,False,False,False,False,True,False,False,False,False
2,True,False,False,False,False,False,False,True,False
3,False,False,False,True,True,True,True,False,False
4,True,False,False,True,True,True,False,True,True


In [25]:
#features.to_csv('features', index = False)

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(features, output[['thermal']], test_size=0.25, random_state=10)

In [27]:
D_train = xgb.DMatrix(X_train, label = Y_train)
D_test = xgb.DMatrix(X_test, label = Y_test)

In [42]:
#best params so far based on grid search
param = {
    'eta': 0.3, 
    'max_depth': 6,  
    'objective': 'multi:softprob',  
    'num_class': 3,
    'gamma' : 0.4,
    'colsample_bytree': 0.7,
    'min_child_weight': 1
    } 

steps = 20  # The number of training iterations

In [43]:
model = xgb.train(param, D_train, steps)

[21:46:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [44]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9555934783228641
Recall = 0.8615703538928472
Accuracy = 0.931


In [31]:
from sklearn.model_selection import GridSearchCV

clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, Y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [-0.33628573 -0.33639484 -0.33638383 ...         nan -0.02437942
 -0.03046172]
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[17:14:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_

In [33]:
grid.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7,
              enable_categorical=False, eta=0.3, gamma=0.4, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [34]:
grid.best_score_

-0.01700347733689701

In [35]:
grid.best_params_

{'colsample_bytree': 0.7,
 'eta': 0.3,
 'gamma': 0.4,
 'max_depth': 6,
 'min_child_weight': 1}